In [ ]:
# solve MIP for graph G with maximal cycle length L

function solve_MIP(nb_max_cycles = 10^6)
    # 'nb_max_cycles' est le nombre maximal de cycles qui peuvent être généré dans la fonction 'simplecycles_limited_length'
    cycles = simplecycles_limited_length(GRAPH, L, nb_max_cycles)
    
    Wc = []
    for c in cycles
        push!(c,c[1]) # ajout du premier sommet du cycle à la fin de la liste pour "faire le tour"
        push!(Wc, sum(WEIGHTS[(c[i],c[i+1])] for i in 1:length(c)-1))
    end
    
    KEP = Model(Gurobi.Optimizer)

    @variable(KEP, z[c in 1:length(cycles)], Bin)
    
    # Contrainte : chaque sommet (i.e. pair donneur/receveur) est choisi une fois au plus
    @constraint(KEP, choosecycle[i in VERTICES], sum(z[c] for c in 1:length(cycles) if i in cycles[c]) <= 1)

    # Objectif : maximiser le score total 
    @objective(KEP, Max, sum(Wc[c]*z[c] for c in 1:length(cycles)))
    
    optimize!(KEP)

    return JuMP.objective_value(KEP), JuMP.value.(z)
end